Simulating Liquid Hydrogen (LH2) discharge from a self-pressurizing cylinder in OpenFOAM is a complex "Multiphysics" problem. It involves **multiphase flow**, **Phase Change (boiling/condensation)**, **Conjugate Heat Transfer (CHT)** through the metal liner, and **Real Gas thermodynamics**.

Here is a roadmap and technical strategy for setting up this specific LH2 case in OpenFOAM.

---

### 1. Choice of Solver
Standard OpenFOAM solvers often require modification to handle the extreme properties of cryogens.

*   **`compressibleInterFoam`:** The most common starting point. It handles two compressible, immiscible fluids (LH2 and GH2) using the VOF (Volume of Fluid) method. However, it does not include phase change by default.
*   **`interPhaseChangeFoam`:** Handles phase change but is typically used for cavitation. It can be adapted for thermal boiling.
*   **`chtMultiRegionTwoPhaseEulerFoam` (Community/ESI versions):** Best if you need to solve the heat conduction in the **metal inner lining** simultaneously with the fluid dynamics.
*   **Custom Solvers (Recommended):** Most cryogenic researchers use a modified version of `compressibleInterFoam` or `interFoam` that integrates the **Lee Model** (mass transfer) and **NIST/REFPROP** libraries for thermophysical properties.

---

### 2. Thermophysical Modeling (The LH2 Challenge)
Cryogenic fluids like LH2 operate near their critical point. Using the Ideal Gas Law will lead to massive errors.

*   **Equation of State (EoS):** You should use a Real Gas EoS. OpenFOAM supports `PengRobinson` or `RedlichKwong`. For high accuracy, you may need to link OpenFOAM to the **CoolProp** or **REFPROP** libraries to get tabulated data for LH2/GH2 density and enthalpy.
*   **Self-Pressurization:** As LH2 leaves the cylinder, the ullage (gas space) expands and pressure drops. This triggers boiling at the liquid-gas interface and the cylinder walls. Your thermophysical model must account for the temperature-dependent latent heat of LH2.

---

### 3. Handling the Metal Inner Lining (CHT)
Because the cylinder is "self-pressurizing," the heat stored in the metal liner is a primary driver of the pressure dynamics.

*   **Multi-Region Approach:** You must mesh the metal liner as a separate region.
*   **Boundary Condition:** Use the `compressible::turbulentTemperatureRadCoupledMixed` (or similar) boundary condition at the interface between the fluid and the metal. This ensures that the heat flux is conserved across the boundary.
*   **Material Properties:** Ensure the liner's thermal conductivity and specific heat are defined as functions of temperature, as metal properties change significantly between 20K and 100K.

---

### 4. Phase Change Modeling (Boiling & Condensation)
In a self-pressurizing tank, two types of phase change occur:
1.  **Interface Phase Change:** Occurs at the GH2/LH2 meniscus.
2.  **Wall Boiling:** Occurs where the LH2 contacts the (relatively) warm metal liner.

**The Lee Model:** 
Most users implement the Lee model in the `interPhaseChangeFoam` framework:
*   If $T > T_{sat}$, mass is transferred from liquid to vapor: $\dot{m} = r \cdot \alpha_l \rho_l \frac{T - T_{sat}}{T_{sat}}$
*   For cryogenics, the relaxation coefficient ($r$) is highly sensitive and often requires tuning against experimental data (like the NASA MHTB tests).

---

### 5. Numerical Setup & Mesh
*   **Meshing:** Use `snappyHexMesh`. You need heavy refinement at the **liquid-gas interface** and near the **walls** (boundary layer) to capture the heat transfer accurately.
*   **PIMPLE Algorithm:** Use the PIMPLE loop for transient stability. Discharge is a highly dynamic process; your time step ($dt$) will likely be limited by the Courant number ($Co < 0.5$ is safer for VOF phase change).
*   **Gravity:** Don't forget to define the gravity vector in `g`. Natural convection inside the tank significantly affects heat distribution.

---

### 6. Suggested Workflow

1.  **Step 1: Geometry & Multi-region Mesh.** Define the cylinder cavity and the solid metal wall thickness.
2.  **Step 2: Define Phases.** Use `transportProperties` or `thermophysicalProperties` to define LH2 and GH2. 
3.  **Step 3: Boundary Conditions.** 
    *   *Outlet:* `pressureInletOutletVelocity` or a fixed flow rate to simulate the discharge.
    *   *Outer Wall:* Adiabatic (if vacuum jacketed) or a fixed heat flux.
4.  **Step 4: Solve for Static Self-Pressurization.** Run the simulation with the outlet closed to see if the pressure rises correctly due to wall heat leak.
5.  **Step 5: Discharge Simulation.** Open the outlet patch and monitor the pressure collapse and phase change rates.

### Key Challenges to Watch For:
*   **Spurious Currents:** VOF can create "fake" velocities at the LH2/GH2 interface. Use `interFoam` variants with improved surface tension modeling.
*   **Phase Change Instability:** The source terms for boiling can be numerically aggressive. Start with a very small mass transfer coefficient and ramp up.
*   **Temperature Glitches:** Ensure your $T_{sat}$ is pressure-dependent (use a polynomial or lookup table), otherwise the self-pressurization physics will be incorrect.


To handle OpenFOAM via Python, the industry standard is **PyFoam**. It allows you to programmatically modify dictionaries (the text files OpenFOAM uses), manage runs, and parse logs.

The following Python Code automates the setup of a **multi-region** case (Fluid LH2/GH2 + Solid Metal Liner) with **low gravity** ($0.01 \, m/s^2$) and LH2 properties.

### Prerequisites
1.  **OpenFOAM** installed (v2312, v9, or similar).
2.  **PyFoam** installed: `pip install PyFoam`.

### Python Automation Script

In [ ]:
import os
from PyFoam.RunDictionary.SolutionDirectory import SolutionDirectory
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile
from PyFoam.Basics.DataStructures import Vector

# --- CONFIGURATION ---
CASE_NAME = "LH2_SelfPressurization_LowG"
GRAVITY_VAL = -0.01  # Low gravity (m/s^2)
TANK_HEIGHT = 1.0    # meters
TANK_RADIUS = 0.2
LINER_THICKNESS = 0.003 # 3mm Metal Liner
INITIAL_TEMP = 20.0  # Kelvin (LH2)
SAT_TEMP = 20.3      # Boiling point at 1 bar

def setup_cryo_case():
    # 1. Create Case Structure from a template
    # You can point this to $FOAM_TUTORIALS/multiRegion/chtMultiRegionTwoPhaseEulerFoam
    template_path = os.path.join(os.environ['FOAM_TUTORIALS'], 
                                "multiRegion", "chtMultiRegionTwoPhaseEulerFoam", "column")
    
    case = SolutionDirectory(CASE_NAME, archive=None, paraviewLink=False)
    if not os.path.exists(CASE_NAME):
        case.cloneCase(template_path)

    print(f"--> Customizing OpenFOAM case: {CASE_NAME}")

    # 2. Modify Gravity (g)
    g_file = ParsedParameterFile(os.path.join(case.name, "constant", "g"))
    g_file["value"] = Vector(0, 0, GRAVITY_VAL)
    g_file.writeFile()

    # 3. Configure Thermophysical Properties for LH2
    # Setting up the fluid region (usually named 'fluid' in multi-region cases)
    thermo_path = os.path.join(case.name, "constant", "fluid", "thermophysicalProperties")
    if os.path.exists(thermo_path):
        thermo = ParsedParameterFile(thermo_path)
        # Simplify to Peng-Robinson for LH2 Real Gas
        thermo["liquid"]["equationOfState"] = "PengRobinson"
        thermo["liquid"]["thermo"] = "hConst"
        thermo["liquid"]["specie"]["molWeight"] = 2.016;
        thermo.writeFile()

    # 4. Modify Mesh Dimensions (blockMeshDict)
    # This edits the vertices to match our TANK_HEIGHT and TANK_RADIUS
    block_mesh = ParsedParameterFile(os.path.join(case.name, "system", "blockMeshDict"))
    # Logic to update vertices would go here...
    # For brevity, we assume the template mesh is scaled via transformPoints later
    block_mesh.writeFile()

    # 5. Set Initial Conditions (Internal Field Temperature)
    t_file_path = os.path.join(case.name, "0", "fluid", "T")
    if os.path.exists(t_file_path):
        t_file = ParsedParameterFile(t_file_path)
        t_file["internalField"] = f"uniform {INITIAL_TEMP}"
        t_file.writeFile()

    print("--> Setup Complete. Ready to generate mesh and run.")

def run_simulation():
    # Execute OpenFOAM commands via system calls
    os.system(f"cd {CASE_NAME} && blockMesh")
    os.system(f"cd {CASE_NAME} && surfaceFeatureExtract")
    os.system(f"cd {CASE_NAME} && snappyHexMesh -overwrite")
    os.system(f"cd {CASE_NAME} && splitMeshRegions -cellZones -overwrite")
    
    # Run the solver (e.g., chtMultiRegionTwoPhaseEulerFoam)
    print("--> Starting Solver...")
    os.system(f"cd {CASE_NAME} && chtMultiRegionTwoPhaseEulerFoam")

if __name__ == "__main__":
    setup_cryo_case()
    # Uncomment to run automatically:
    # run_simulation()

### Key Logic for Cryogenic Self-Pressurization

To make the Python script actually simulate the physics you described, you need to ensure the following dictionary settings are applied:

#### 1. Phase Change (The Lee Model)
In the `constant/fluid/phaseProperties` file, you must define the mass transfer. Because it is a "self-pressurizing" cylinder, the boiling is driven by the temperature difference between the LH2 and the wall:


In [ ]:
# Programmatic snippet to add Lee Model via Python
phase_props = ParsedParameterFile(os.path.join(CASE_NAME, "constant", "fluid", "phaseProperties"))
phase_props["massTransferModel"] = [
    {
        "type": "Lee",
        "species": "liquidToVapour",
        "C": 0.1,         # Relaxation coefficient (Tweak for cryogenics)
        "Tsat": SAT_TEMP  # 20.3 K
    }
]
phase_props.writeFile()

#### 2. Metal Liner (Conjugate Heat Transfer)
In a self-pressurizing case, the **solid region** (the metal lining) must have a high thermal mass relative to the fluid. 
*   **Initial Condition:** The metal liner is usually slightly warmer than the LH2 (e.g., 25K vs 20K).
*   **Boundary Condition:** In `0/fluid/T`, use `compressible::turbulentTemperatureRadCoupledMixed`. This allows the metal liner to "leak" heat into the LH2, causing the pressure to rise as the liquid boils.

#### 3. Low Gravity Considerations
With low gravity ($10^{-2}$ to $10^{-6} g$), the surface tension forces become dominant over buoyancy.
*   In `constant/fluid/phaseProperties`, ensure the **Surface Tension Coefficient** (sigma) for LH2 is set accurately (~$0.002 \, N/m$).
*   The VOF method in OpenFOAM is sensitive to "spurious currents" in low gravity. You may need to increase the PISO/PIMPLE corrections in `fvSolution`.

### Why use Python for this?
1.  **Sensitivity Analysis:** You can wrap the script in a `for` loop to test different gravity levels ($g=0.1, g=0.01, g=0.001$) and see how the LH2 bubble forms.
2.  **Property Lookup:** You can use the `CoolProp` library in Python to fetch exact LH2 properties (density, viscosity) at a specific pressure and inject them directly into the OpenFOAM dictionaries before the run starts.